In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy import signal
import sys

In [3]:
visitnum = ['FirstVisit/', 'Later7thVisit/','LaterVisit/', 'LastVisit/']

In [4]:
for visit in visitnum[0:1]:
    direc = os.path.join(r"../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/CleanedUp/",
                         visit)

#     videoGuide = pd.read_csv(glob.glob(direc + "AllVideoNames.csv")[0])

    trackslist = glob.glob(direc + 'c*.csv')
    
#     Manual_direc = os.path.join(r"../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/CleanedUp/",
#                                visit)
#     cleaneduplist = glob.glob(Manual_direc + 'c*.csv')

In [5]:
newpath = os.path.join(r"../dataFolders/PaperPipelineOutput/RadiusAndAngle_v2/CleanedUp", visit)
if not os.path.exists(newpath):
    os.mkdir(newpath)

In [6]:
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

## get angle and magnitude

In [7]:
def Unitvector(x_gauss, y_gauss):
    from sklearn import preprocessing
    
    # get the slope of the tangent
    trajectory = np.asarray([x_gauss, y_gauss])
    m = np.gradient(trajectory, axis = 1)
    m_atx = m[1]/m[0]
    
    # get the tangent vector at x = x0 + 1
    tangent_x = x_gauss+1
    tangent_y = m_atx + y_gauss
    
    # get the unit tangent vector
    u_x = []
    u_y = []

    for x,y,x0,y0 in zip(tangent_x, tangent_y, x_gauss, y_gauss):
        if np.any(np.isnan([x, y])) or np.any(np.isinf([x, y])):
            unit_x = np.nan
            unit_y = np.nan
        else:
            vector = np.asarray([x-x0, y-y0]).reshape(1,-1)
            [unit_x, unit_y] = preprocessing.normalize(vector, norm = 'l2')[0]
        u_x.append(unit_x)
        u_y.append(unit_y)
    u_x = np.asarray(u_x)
    u_y = np.asarray(u_y)
    return(u_x, u_y)

def getAngle(loc, tangent):
    cross = np.cross(tangent, loc)
    dot = np.dot(tangent, loc)
    angle = np.arctan2(cross, dot)*180/np.pi
    return(angle)

def wrapAngle(angle):
    angle = np.absolute(angle)
    for i,a in enumerate(angle):
        if a > 90:
            a = 180 - a
            angle[i] = a
    return(angle)

In [15]:
file = pd.read_csv(trackslist[0])
test = file.dropna()
test.columns = ['x','y']
test.head()

,x,y
9915,329.140028,303.905828
9916,327.291653,287.410417
9917,316.201400,293.080714
9918,307.382181,299.138978
9919,310.788768,313.268657


In [16]:
for data in trackslist:
#     data = trackslist[0]
    moth = os.path.basename(data)[:-4]
    
    segments = moth.split("_")
    moth = segments[0] + '_' + segments[1]
    
#     selected = videoGuide.loc[videoGuide.AutomatatedTracking == 'TRUE', 'names'].values
    
#     if moth not in selected:
#         continue
#     if not [test for test in cleaneduplist if moth in test]:
#         continue
            
    file = pd.read_csv(data)
    # for manually edited tracks - add the next few lines
    
    file = file.dropna()
    file.columns = ['x', 'y']
    
    x = file.x.values
    y = file.y.values
    r = file.r.values
    
    name = [n for n in circ_parameters.name if n + '_' in data][0]

    circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
    circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
    circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values
    
    x = x - circ_x
    y = y - circ_y
    
    # save all usable variables as series

    df1 = pd.Series(data = x, name = 'x_centered')
    df2 = pd.Series(data = y, name = 'y_centered')
    df3 = pd.Series(data = r, name = 'radial_distance_normalized')


    #calculate the unit tangent vectors - savitzky-golay vector
    u_x, u_y = Unitvector(x, y)

    angle = []
    for x0, y0, x, y in zip(x, y, u_x, u_y):
        loc = [x0, y0]
        tangent = [x, y]
        a = getAngle(loc, tangent)
        angle.append(a)

    angle = wrapAngle(angle)

    df4 = pd.Series(data = angle, name = 'angle')

    new_file = pd.concat([df1, df2, df3, df4], axis = 1)
    new_file.to_csv(newpath + moth + '_RadiusAndAngle.csv', index_label = False)

AttributeError: 'DataFrame' object has no attribute 'r'